In [ ]:
# ------------------------------------------------------------
# # This script prints every EPL match ID (with the fixture text) that falls within a given date window of the 2025/26 season, and then attempts to pull shot events based on xG.
#
# ✅ What you edit:
#   - Change START_DATE and END_DATE below (YYYY-MM-DD).
#   - Optionally change LEAGUE and SEASON if you want a different league/season.
#
# ▶ How it works (briefly):
#   - Fetch weekly league pages, decode `datesData = JSON.parse('...')`,
#     collect fixtures, filter by your date window, de-duplicate by match_id,
#     and print `ID <match_id> — <Home> v <Away> (<kickoff>)`.
#   - For shots: attempts to read `shotsData` for each match page.
#     (Future matches will naturally fail because shots don’t exist yet.)
# ------------------------------------------------------------

from __future__ import annotations

import os
import re
import json
import time
import unicodedata
from typing import Optional

import requests
import pandas as pd

# ======== EDIT THESE ========
LEAGUE = "EPL"
SEASON = 2025
START_DATE = "2025-08-15"
END_DATE   = "2025-08-15"

# Save per-match shots CSVs
SAVE_CSV = True
OUT_DIR = r"C:\Users\jseat\projects\LiverpoolFC\notebooks\data\raw"
# ============================

BASE = "https://understat.com"
S = requests.Session()
S.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
})

# ---------------------------
# Generic helpers
# ---------------------------

def _fname_for_match(home: str, away: str, ko_ts) -> str:
    """Filesystem-safe, readable filename."""
    def slug(x: str) -> str:
        # letters/numbers -> keep; everything else -> underscore; collapse repeats
        return re.sub(r'[^A-Za-z0-9]+', '_', x).strip('_')
    return f"{slug(home)}_v_{slug(away)}_{ko_ts:%Y-%m-%d}_shots.csv"

def _decode_js_escaped(s: str) -> str:
    """Decode \\xNN / \\uXXXX style escapes from JSON.parse('...') payloads."""
    return bytes(s, "utf-8").decode("unicode_escape")

def _extract_array(text: str) -> str:
    """Quote/escape-aware bracket scan that returns the JSON array substring."""
    i = text.find("[")
    if i == -1:
        raise RuntimeError("No '[' found in decoded datesData.")
    out, depth, esc, in_str, quote = [], 0, False, False, ""
    j = i
    while j < len(text):
        ch = text[j]
        out.append(ch)
        if in_str:
            if esc:
                esc = False
            elif ch == "\\":
                esc = True
            elif ch == quote:
                in_str = False
        else:
            if ch in ('"', "'"):
                in_str = True; quote = ch
            elif ch == "[":
                depth += 1
            elif ch == "]":
                depth -= 1
                if depth == 0:
                    break
        j += 1
    if depth != 0:
        raise RuntimeError("Unbalanced brackets while slicing datesData.")
    return "".join(out)

def _jsonparse_object(page_text: str, varname: str):
    """Extracts JSON.parse('...') object assigned to varname, returns Python object or None."""
    m = re.search(rf"{re.escape(varname)}\s*=\s*JSON\.parse\('(.*?)'\);", page_text, flags=re.S)
    if not m:
        return None
    decoded = _decode_js_escaped(m.group(1))
    i = decoded.find("{")
    if i == -1:
        return None
    out, depth, in_str, esc, quote = [], 0, False, False, ""
    for ch in decoded[i:]:
        out.append(ch)
        if in_str:
            if esc: esc = False
            elif ch == "\\": esc = True
            elif ch == quote: in_str = False
        else:
            if ch in ("'", '"'): in_str = True; quote = ch
            elif ch == "{": depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0: break
    try:
        return json.loads("".join(out))
    except Exception:
        return None

def _jsonparse_array(page_text: str, varname: str):
    """Extracts JSON.parse('...') array assigned to varname, returns Python list or None."""
    m = re.search(rf"{re.escape(varname)}\s*=\s*JSON\.parse\('(.*?)'\);", page_text, flags=re.S)
    if not m:
        return None
    decoded = _decode_js_escaped(m.group(1))
    i = decoded.find("[")
    if i == -1:
        return None
    out, depth, in_str, esc, quote = [], 0, False, False, ""
    for ch in decoded[i:]:
        out.append(ch)
        if in_str:
            if esc: esc = False
            elif ch == "\\": esc = True
            elif ch == quote: in_str = False
        else:
            if ch in ("'", '"'): in_str = True; quote = ch
            elif ch == "[": depth += 1
            elif ch == "]":
                depth -= 1
                if depth == 0: break
    try:
        return json.loads("".join(out))
    except Exception:
        return None

def _norm(s: Optional[str]) -> str:
    """ASCII-fold, drop diacritics, keep alnum/space, collapse whitespace, lowercase."""
    if s is None:
        return ""
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = "".join(ch if ch.isalnum() or ch.isspace() else " " for ch in s)
    return " ".join(s.lower().split())

def _slug(name: str) -> str:
    """Best-effort slug for team-season URL."""
    special = {
        "brighton": "Brighton",
        "west_ham": "West_Ham",
        "nottingham_forest": "Nottingham_Forest",
        "crystal_palace": "Crystal_Palace",
        "manchester_city": "Manchester_City",
        "manchester_united": "Manchester_United",
        "aston_villa": "Aston_Villa",
        "newcastle_united": "Newcastle_United",
        "wolverhampton_wanderers": "Wolverhampton_Wanderers",
        "leeds_united": "Leeds_United",
    }
    key = _norm(name).replace(" ", "_")
    return special.get(key, key.title().replace("_", "_"))

# ---------------------------
# Fixtures (league week pages)
# ---------------------------

def _fetch_week_fixtures(league: str, season: int, week: int) -> pd.DataFrame:
    """Return a DataFrame of fixtures for a given week (or empty DF if none)."""
    url = f"{BASE}/league/{league}/{season}?week={week}"
    html = S.get(url, timeout=30)
    html.raise_for_status()
    text = html.text

    # A) datesData = JSON.parse(' ... ');
    m = re.search(r"datesData\s*=\s*JSON\.parse\('(.*?)'\);", text, flags=re.S)
    if m:
        decoded = _decode_js_escaped(m.group(1))
        arr_txt = _extract_array(decoded)
        data = json.loads(arr_txt)
    else:
        # B) Fallback: datesData = [ ... ];
        m2 = re.search(r"datesData\s*=\s*(\[[\s\S]*?\]);", text, flags=re.S)
        if not m2:
            return pd.DataFrame()  # week page without fixtures
        data = json.loads(m2.group(1))

    df = pd.json_normalize(data)

    # kickoff datetime (make tz-naive)
    for col in ("datetime", "date", "time"):
        if col in df.columns:
            df["kickoff"] = pd.to_datetime(df[col], errors="coerce")
            break
    else:
        df["kickoff"] = pd.NaT
    df["kickoff"] = df["kickoff"].dt.tz_localize(None)

    # Normalize a few common fields
    if "id" in df.columns:
        df["match_id"] = df["id"].astype("Int64")
    if "h.title" in df.columns:
        df.rename(columns={"h.title": "home_team", "a.title": "away_team"}, inplace=True)

    return df[["match_id", "home_team", "away_team", "kickoff"]].copy()

def _collect_fixtures_in_window(league: str, season: int, start_date: str, end_date: str) -> pd.DataFrame:
    """Scan weeks until safely past the end of the window; return filtered+deduped DF."""
    start = pd.to_datetime(start_date)
    end_exclusive = pd.to_datetime(end_date) + pd.Timedelta(days=1)  # inclusive end
    all_rows = []

    # August usually spans weeks 1–3; keep a small safe range.
    for week in range(1, 6):
        df = _fetch_week_fixtures(league, season, week)
        if df.empty:
            continue
        all_rows.append(df)
        # Early stop if clearly beyond the window
        wk_min = df["kickoff"].min()
        if pd.notna(wk_min) and wk_min > end_exclusive + pd.Timedelta(days=7):
            break

    if not all_rows:
        return pd.DataFrame(columns=["match_id", "home_team", "away_team", "kickoff"])

    fixtures = pd.concat(all_rows, ignore_index=True)

    # Filter to window
    in_window = fixtures[(fixtures["kickoff"] >= start) & (fixtures["kickoff"] < end_exclusive)]

    # De-duplicate by match_id
    in_window = in_window.drop_duplicates(subset=["match_id"]).sort_values("kickoff").reset_index(drop=True)
    return in_window

# ---------------------------
# Shots (match pages)
# ---------------------------

def _fetch_shots_by_match_id(match_id: int):
    """Fetch raw shotsData object (dict with 'h' and 'a') for a single match_id; None if missing."""
    url = f"{BASE}/match/{match_id}"
    text = S.get(url, timeout=30).text

    shots = _jsonparse_object(text, "shotsData")
    if shots is None:
        # fallback: plain object assignment
        m2 = re.search(r"shotsData\s*=\s*({[\s\S]*?});", text, flags=re.S)
        if m2:
            try:
                shots = json.loads(m2.group(1))
            except Exception:
                shots = None
    return shots

def fetch_shots_df(match_id: int) -> pd.DataFrame:
    """Return a tidy DataFrame of all shot events for a match_id (raises if not found)."""
    shots = _fetch_shots_by_match_id(match_id)
    if shots is None:
        raise RuntimeError(f"shotsData not found for match {match_id}")

    rows = []
    for side in ("h", "a"):
        for ev in shots.get(side, []):
            rows.append({
                "match_id": match_id,
                "side": side,
                "team": ev.get("team"),
                "player": ev.get("player"),
                "player_id": ev.get("player_id"),
                "minute": ev.get("minute"),
                "second": ev.get("second"),
                "xG": ev.get("xG"),
                "result": ev.get("result"),
                "situation": ev.get("situation"),
                "shotType": ev.get("shotType"),
                "X": ev.get("X"),
                "Y": ev.get("Y"),
                "assist": ev.get("assist"),
                "assist_id": ev.get("assist_id"),
                "lastAction": ev.get("lastAction"),
            })
    return pd.DataFrame(rows).sort_values(["side", "minute", "second"], na_position="last").reset_index(drop=True)

# ---------------------------
# Optional helper used by the demo to resolve a "true" match id
# from team pages (kept as in your original code).
# ---------------------------

def _resolve_true_match_id_from_team(home: str, away: str, ko, season: int) -> Optional[int]:
    for team in (home, away):
        slug = _slug(team)
        url = f"{BASE}/team/{slug}/{season}"
        text = S.get(url, timeout=30).text

        for var in ("matchesData", "matches"):
            arr = _jsonparse_array(text, var)
            if not isinstance(arr, list):
                continue
            target = {_norm(home), _norm(away)}
            best, best_dt = None, pd.Timedelta(days=365)
            for row in arr:
                if not isinstance(row, dict):
                    continue
                h = (row.get("h", {}) or {}).get("title") or row.get("h.title")
                a = (row.get("a", {}) or {}).get("title") or row.get("a.title")
                if not h or not a:
                    continue
                if {_norm(h), _norm(a)} != target:
                    continue
                dt = row.get("datetime") or (f"{row.get('date','')} {row.get('time','')}".strip() or None)
                t = pd.to_datetime(dt, errors="coerce")
                if pd.isna(t):
                    continue
                t = t.tz_localize(None)
                diff = abs(t - ko)
                if diff < best_dt:
                    best_dt, best = diff, row.get("id")
            if best:
                return int(best)
    return None

# ---------------------------
# Runners
# ---------------------------

def print_fixtures_in_window():
    """Print fixtures and IDs within the date window."""
    win = _collect_fixtures_in_window(LEAGUE, SEASON, START_DATE, END_DATE)
    if win.empty:
        print(f"No fixtures found between {START_DATE} and {END_DATE}.")
        return
    for _, r in win.iterrows():
        mid  = int(r["match_id"])
        home = r.get("home_team", "")
        away = r.get("away_team", "")
        ko   = r["kickoff"].strftime("%Y-%m-%d %H:%M")
        print(f"ID {mid} — {home} v {away} ({ko})")

def demo_one():
    """Minimal demo on the first match in window (unchanged behavior)."""
    win = _collect_fixtures_in_window(LEAGUE, SEASON, START_DATE, END_DATE)
    if win.empty:
        print("No fixtures in window.")
        return
    r = win.iloc[0]
    mid, home, away, ko = int(r["match_id"]), r["home_team"], r["away_team"], r["kickoff"]
    print(f"Fixture: ID {mid} — {home} v {away} ({ko:%Y-%m-%d %H:%M})")

    # 1) Try using the small ID directly
    shots = _fetch_shots_by_match_id(mid)
    if shots:
        print("✔ shotsData found using this ID (great!)")
    else:
        print("✖ shotsData NOT found with this ID; resolving true Understat match id…")
        true_id = _resolve_true_match_id_from_team(home, away, ko, SEASON)
        if not true_id:
            print("Could not resolve a true Understat match id from team pages.")
            return
        print(f"Resolved true match id: {true_id}")
        shots = _fetch_shots_by_match_id(true_id)
        if not shots:
            print("Even the true id returned no shotsData; page structure may have changed.")
            return

    # Print a few events
    rows = []
    for side in ("h", "a"):
        for ev in shots.get(side, []):
            rows.append({
                "side": side,
                "team": ev.get("team"),
                "player": ev.get("player"),
                "minute": ev.get("minute"),
                "xG": ev.get("xG"),
                "result": ev.get("result"),
            })
    df = pd.DataFrame(rows).sort_values(["side", "minute"]).reset_index(drop=True)
    print(f"Pulled {len(df)} shot events; showing first 5:")
    print(df.head(5))

def fetch_all_matches_in_window():
    """Iterate all matches in the window and attempt to fetch shot events."""
    win = _collect_fixtures_in_window(LEAGUE, SEASON, START_DATE, END_DATE)
    if win.empty:
        print(f"No fixtures found between {START_DATE} and {END_DATE}.")
        return

    print(f"Found {len(win)} matches in window.")
    for _, r in win.iterrows():
        mid  = int(r["match_id"])
        home = r.get("home_team", "")
        away = r.get("away_team", "")
        ko   = r["kickoff"].strftime("%Y-%m-%d %H:%M")
        print(f"→ ID {mid} — {home} v {away} ({ko})")

        try:
            df = fetch_shots_df(mid)
        except Exception as e:
            print(f"   △ failed to fetch shots: {e}")
            continue

        print(f"   ✓ {len(df)} shots")
        if SAVE_CSV:
            os.makedirs(OUT_DIR, exist_ok=True)
            out_name = _fname_for_match(home, away, r["kickoff"])
            out_path = os.path.join(OUT_DIR, out_name)
            df.to_csv(out_path, index=False, encoding="utf-8-sig")
            print(f"   ↳ saved {out_path}")

        time.sleep(0.6)  # be polite

# ---------------------------
# Main
# ---------------------------

if __name__ == "__main__":
    # 1) Print fixture IDs in window
    print_fixtures_in_window()

    # 2) Minimal one-match demo (as in your original code)
    demo_one()

    # 3) Attempt shots for all matches in the window
    fetch_all_matches_in_window()

ID 28778 — Liverpool v Bournemouth (2025-08-15 19:00)
Fixture: ID 28778 — Liverpool v Bournemouth (2025-08-15 19:00)
✔ shotsData found using this ID (great!)
Pulled 29 shot events; showing first 5:
  side  team            player minute                    xG       result
0    a  None  Marcus Tavernier     34   0.35100075602531433    SavedShot
1    a  None      David Brooks     41  0.038630276918411255  BlockedShot
2    a  None  Marcus Tavernier     46  0.045542508363723755  BlockedShot
3    a  None         Evanilson     48   0.09731415659189224  MissedShots
4    a  None   Antoine Semenyo      5    0.3169209659099579  MissedShots
Found 1 matches in window.
→ ID 28778 — Liverpool v Bournemouth (2025-08-15 19:00)
   ✓ 29 shots
   ↳ saved C:\Users\jseat\projects\LiverpoolFC\notebooks\data\raw\Liverpool_v_Bournemouth_2025-08-15_shots.csv
